# Shopee Orders Data Extraction

Notebook này thực hiện việc lấy dữ liệu đơn hàng từ Shopee API, từ bước lấy access token đến xử lý và xuất dữ liệu theo cấu trúc phân nhánh.

## Cấu trúc dữ liệu:
- **orders**: Thông tin chính của đơn hàng (1-1 theo order_sn)
- **recipient_address**: Địa chỉ người nhận (0-1/1-1 theo order_sn)
- **order_items**: Danh sách sản phẩm trong đơn hàng (1-n theo order_sn)
- **packages**: Thông tin kiện hàng (0-n theo order_sn)
- **package_items**: Sản phẩm trong từng kiện hàng (1-n theo package)
- **invoice**: Thông tin hóa đơn (0-1 theo order_sn)
- **payment_info**: Thông tin thanh toán bổ sung (0-n theo order_sn)
- **pending_terms**: Điều khoản chờ xử lý (0-n theo order_sn)
- **warnings**: Cảnh báo (0-n theo order_sn)
- **prescription_images**: Ảnh đơn thuốc (0-n theo order_sn)


In [7]:
# Cài đặt các thư viện cần thiết
%pip install requests pandas openpyxl

import requests
import hmac
import hashlib
import time
import pandas as pd
import json
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
import warnings
warnings.filterwarnings('ignore')


Note: you may need to restart the kernel to use updated packages.


## 1. Cấu hình thông tin xác thực


In [6]:
# Thông tin xác thực Shopee API
PARTNER_ID = '2012924'  # Thay thế bằng Partner ID thực tế
PARTNER_KEY = 'shpk614c4b47456867715a6c6867456751514f6e724b7a64574468756f7a7a6c'  # Thay thế bằng Partner Key thực tế
SHOP_ID = '1286901283'  # Thay thế bằng Shop ID thực tế
REDIRECT_URI = 'https://facolospickleball.com/'  # Thay thế bằng Redirect URI thực tế

print(f"Partner ID: {PARTNER_ID}")
print(f"Shop ID: {SHOP_ID}")
print(f"Redirect URI: {REDIRECT_URI}")


Partner ID: 2012924
Shop ID: 1286901283
Redirect URI: https://facolospickleball.com/


## 2. Hàm tiện ích để tạo chữ ký API


In [5]:
def create_signature(partner_id: str, partner_key: str, path: str, timestamp: int, access_token: str = None, shop_id: str = None) -> str:
    """
    Tạo chữ ký cho API request theo chuẩn Shopee
    """
    if access_token and shop_id:
        base_string = f"{partner_id}{path}{timestamp}{access_token}{shop_id}"
    else:
        base_string = f"{partner_id}{path}{timestamp}"
    
    signature = hmac.new(
        partner_key.encode('utf-8'),
        base_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()
    
    return signature


## 3. Lấy Authorization Code

Bước 1: Truy cập URL sau để lấy authorization code (có thêm timestamp và sign):
```
https://partner.shopeemobile.com/api/v2/shop/auth_partner?partner_id=YOUR_PARTNER_ID&redirect=YOUR_REDIRECT_URI&timestamp=TIMESTAMP&sign=SIGN
```

**Lưu ý quan trọng**: URL phải bao gồm cả `timestamp` và `sign` để tránh lỗi "no timestamp". Sử dụng cell bên dưới để tạo URL chính xác.

Sau khi ủy quyền, bạn sẽ được chuyển hướng đến redirect URI với tham số `code` trong URL. Lưu lại giá trị của `code` này.


In [8]:

# Tạo URL để lấy authorization code với timestamp và sign
def create_auth_url(partner_id: str, partner_key: str, redirect_uri: str) -> str:
    """
    Tạo URL authorization với timestamp và sign để tránh lỗi 'no timestamp'
    """
    path = '/api/v2/shop/auth_partner'
    timestamp = int(time.time())
    sign = create_signature(partner_id, partner_key, path, timestamp)
    
    auth_url = f"https://partner.shopeemobile.com{path}?partner_id={partner_id}&redirect={redirect_uri}&timestamp={timestamp}&sign={sign}"
    return auth_url

# Tạo URL authorization chính xác
auth_url = create_auth_url(PARTNER_ID, PARTNER_KEY, REDIRECT_URI)
print(f"Truy cập URL sau để lấy authorization code:")
print(auth_url)
print(f"\nTimestamp: {int(time.time())}")
print(f"Sign: {create_signature(PARTNER_ID, PARTNER_KEY, '/api/v2/shop/auth_partner', int(time.time()))}")
print("\nSau khi ủy quyền, lưu lại giá trị 'code' từ URL redirect")


Truy cập URL sau để lấy authorization code:
https://partner.shopeemobile.com/api/v2/shop/auth_partner?partner_id=2012924&redirect=https://facolospickleball.com/&timestamp=1760834160&sign=6206dc87d247b3120f8935e5437f5cab79c4cdd3a783679b2926d379bd0c8ba8

Timestamp: 1760834160
Sign: 6206dc87d247b3120f8935e5437f5cab79c4cdd3a783679b2926d379bd0c8ba8

Sau khi ủy quyền, lưu lại giá trị 'code' từ URL redirect


## 4.1 Lấy Access Token từ Authorization Code


In [9]:
def get_access_token(partner_id: str, partner_key: str, code: str, shop_id: str) -> Dict[str, Any]:
    """
    Lấy access token và refresh token từ authorization code
    """
    path = '/api/v2/auth/token/get'
    timestamp = int(time.time())
    
    # Tạo base string cho chữ ký: partner_id + path + timestamp
    base_string = str(partner_id) + path + str(timestamp)
    
    # Tạo chữ ký HMAC-SHA256
    sign = hmac.new(
        partner_key.encode('utf-8'),
        base_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()
    
    url = f"https://partner.shopeemobile.com{path}"
    
    # Query parameters (phải nằm trong URL)
    query_params = {
        'partner_id': int(partner_id),
        'timestamp': timestamp,
        'sign': sign
    }
    
    # Request body (JSON)
    request_body = {
        "shop_id": int(shop_id),
        "code": code,
        "partner_id": int(partner_id)
    }
    
    try:
        response = requests.post(
            url,
            params=query_params,
            headers={'Content-Type': 'application/json'},
            data=json.dumps(request_body)
        )
        
        response_json = response.json()
        
        if response.status_code == 200 and not response_json.get('error'):
            print("✅ LẤY ACCESS TOKEN THÀNH CÔNG!")
            print(f"Access Token: {response_json.get('access_token')}")
            print(f"Refresh Token: {response_json.get('refresh_token')}")
            print(f"Hết hạn sau (giây): {response_json.get('expire_in')}")
            return response_json
        else:
            print(f"❌ LỖI GỌI API ({response.status_code}):")
            print(f"Error Code: {response_json.get('error')}")
            print(f"Message: {response_json.get('message')}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ LỖI KẾT NỐI: {e}")
        return None

# Thay thế 'your_authorization_code' bằng mã bạn đã lấy được
AUTHORIZATION_CODE = '796c4249534d72416144426172556577'  # Thay thế bằng authorization code thực tế

if AUTHORIZATION_CODE != 'your_authorization_code':
    print("Đang lấy access token...")
    token_response = get_access_token(PARTNER_ID, PARTNER_KEY, AUTHORIZATION_CODE, SHOP_ID)
    
    if token_response and not token_response.get('error'):
        access_token = token_response.get('access_token')
        refresh_token = token_response.get('refresh_token')
        print(f"\n✅ Thành công!")
        print(f"Access Token: {access_token}")
        print(f"Refresh Token: {refresh_token}")
    else:
        print("❌ Không thể lấy access token. Vui lòng kiểm tra lại thông tin.")
        access_token = None
        refresh_token = None
else:
    print("Vui lòng thay thế AUTHORIZATION_CODE bằng mã thực tế")
    access_token = None
    refresh_token = None


Đang lấy access token...
✅ LẤY ACCESS TOKEN THÀNH CÔNG!
Access Token: 5857685552434e5148766c756a544577
Refresh Token: 416246786e506b456145506e72556c66
Hết hạn sau (giây): 14386

✅ Thành công!
Access Token: 5857685552434e5148766c756a544577
Refresh Token: 416246786e506b456145506e72556c66


## 4.2 Lấy Access và Refresh Token mới từ Refresh Token


In [6]:
import requests
import time
import hmac
import hashlib
import json

# ===============================================
# ⚠️ THÔNG SỐ CẦN THAY THẾ CỦA BẠN ⚠️
# ===============================================
Refresh_Token = "466b774c584a75536e51497a41786244"
# 1. Partner ID và Key (Lấy từ Shopee Open Platform Console)
PARTNER_ID = 2012924 # Thay bằng Partner ID của bạn (ví dụ: 10001)
PARTNER_KEY = 'shpk614c4b47456867715a6c6867456751514f6e724b7a64574468756f7a7a6c' # Thay bằng Partner Key (Secret Key) của bạn

# # 2. Thông số Ủy quyền (Lấy từ URL Redirect sau khi Seller ủy quyền)
SHOP_ID = 1286901283 # Shop ID từ URL Redirect hoặc Shop ID cần ủy quyền

# 3. URL
# Dùng URL chính thức (hoặc URL khu vực/Test URL nếu cần)
HOST = 'https://partner.shopeemobile.com'
API_PATH = '/api/v2/auth/access_token/get'
URL = HOST + API_PATH

# ===============================================

def generate_sign_and_request():
    """Tạo chữ ký và gửi yêu cầu POST để lấy access_token/refresh_token."""

    # 1. Tạo Timestamp hiện tại (Unix time in seconds)
    timestamp = int(time.time())

    # 2. Tạo Base String cho chữ ký
    # Cấu trúc: Partner ID + API Path + Timestamp
    base_string = str(PARTNER_ID) + API_PATH + str(timestamp)
    # print(f"Base String: {base_string}")

    # 3. Tính toán Chữ ký (Sign)
    # HMAC-SHA256, đầu ra là hex-encoded.
    sign = hmac.new(
        PARTNER_KEY.encode('utf-8'),
        base_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    # print(f"Sign: {sign}")

    # 4. Chuẩn bị Yêu cầu

    # A. Query Parameters (Gắn vào URL)
    # Shopee yêu cầu partner_id, timestamp, và sign phải nằm trong URL
    query_params = {
        'partner_id': PARTNER_ID,
        'timestamp': timestamp,
        'sign': sign
    }

    # B. Request Body (JSON Body)
    # Chứa các thông số chính để lấy token
    request_body = {
        "shop_id": SHOP_ID,
        "refresh_token": Refresh_Token,
        "partner_id": PARTNER_ID
    }

    # 5. Gửi Yêu cầu POST
    try:
        response = requests.post(
            URL,
            params=query_params,
            headers={'Content-Type': 'application/json'},
            data=json.dumps(request_body)
        )

        response_json = response.json()

        if response.status_code == 200 and not response_json.get('error'):
            print("\n✅ LẤY ACCESS TOKEN THÀNH CÔNG!")
            print("====================================")
            print(f"Access Token: {response_json.get('access_token')}")
            print(f"Refresh Token: {response_json.get('refresh_token')}")
            print(f"Hết hạn sau (giây): {response_json.get('expire_in')}")
            print("====================================")
            print("Lưu ý: Access Token hết hạn sau 4 giờ. Hãy dùng Refresh Token để làm mới!")
        else:
            print(f"\n❌ LỖI GỌI API ({response.status_code}):")
            print("====================================")
            print(f"Error Code: {response_json.get('error')}")
            print(f"Message: {response_json.get('message')}")
            print("====================================")

    except requests.exceptions.RequestException as e:
        print(f"\n❌ LỖI KẾT NỐI: {e}")

# Chạy hàm để lấy token
generate_sign_and_request()


❌ LỖI GỌI API (403):
Error Code: error_param
Message: Your refresh token or shop_id is wrong, please check refresh token or shop_id.


## 5. Lấy danh sách đơn hàng của ngày trước


In [26]:
def get_orders(partner_id: str, partner_key: str, shop_id: str, access_token: str, 
              time_from: int, time_to: int, page_size: int = 100, time_range_field: str = "create_time") -> Dict[str, Any]:
    """
    Lấy danh sách đơn hàng trong khoảng thời gian chỉ định. Tuân thủ chuẩn Shopee:
    - partner_id, timestamp, sign, access_token, shop_id phải ở query string
    - sign = HMAC-SHA256(partner_key, partner_id + path + timestamp + access_token + shop_id)
    """
    path = '/api/v2/order/get_order_list'
    timestamp = int(time.time())
    sign = create_signature(str(partner_id), partner_key, path, timestamp, access_token, str(shop_id))

    url = f"https://partner.shopeemobile.com{path}"

    # Tham số bắt buộc trong query string
    params = {
        "partner_id": int(partner_id),
        "timestamp": timestamp,
        "sign": sign,
        "access_token": access_token,
        "shop_id": int(shop_id),
        # Bộ lọc thời gian và phân trang
        "time_range_field": time_range_field,
        "time_from": int(time_from),
        "time_to": int(time_to),
        "page_size": int(page_size)
    }

    # order_status: chỉ thêm khi cung cấp giá trị hợp lệ (UNPAID, READY_TO_SHIP, SHIPPED, COMPLETED, CANCELLED, IN_CANCEL, TO_RETURN, INVOICE_PENDING)
    # Nếu muốn lấy tất cả trạng thái, KHÔNG gửi tham số này
    # Ví dụ: params["order_status"] = "READY_TO_SHIP"

    try:
        response = requests.get(url, params=params)
        data = response.json()
        if response.status_code != 200 or data.get('error'):
            print("❌ Lỗi lấy đơn hàng:")
            print(f"  HTTP {response.status_code}")
            print(f"  error: {data.get('error')}")
            print(f"  message: {data.get('message')}")
            return None
        return data
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi lấy danh sách đơn hàng: {e}")
        return None

# Tính toán thời gian cho ngày trước đó
yesterday = datetime.now() - timedelta(days=1)
time_from = int(yesterday.replace(hour=0, minute=0, second=0).timestamp())
time_to = int(yesterday.replace(hour=23, minute=59, second=59).timestamp())

print(f"Lấy đơn hàng từ: {datetime.fromtimestamp(time_from)}")
print(f"Đến: {datetime.fromtimestamp(time_to)}")

if access_token:
    print("\nĐang lấy danh sách đơn hàng...")
    orders_data = get_orders(PARTNER_ID, PARTNER_KEY, SHOP_ID, access_token, time_from, time_to)

    # Bước debug: In ra cấu trúc JSON trả về để kiểm tra
    if orders_data:
        try:
            print("\n=== DEBUG: CẤU TRÚC PHẢN HỒI GET_ORDER_LIST ===")
            print(f"Khóa cấp 1: {list(orders_data.keys())}")
            preview = json.dumps(orders_data, indent=2, ensure_ascii=False)
            print(preview[:8000])  # Giới hạn hiển thị để tránh quá dài
            with open('debug_get_order_list.json', 'w', encoding='utf-8') as f:
                f.write(preview)
            print("(Đã lưu debug JSON vào debug_get_order_list.json)")
        except Exception as e:
            print(f"DEBUG JSON error: {e}")

    if orders_data and 'response' in orders_data:
        order_list = orders_data['response'].get('order_list', [])
        print(f"Tìm thấy {len(order_list)} đơn hàng")
        
        if order_list:
            print("\nMẫu đơn hàng đầu tiên:")
            print(json.dumps(order_list[0], indent=2, ensure_ascii=False))
    else:
        print("Không thể lấy dữ liệu đơn hàng")
        order_list = []
else:
    print("Không có access token. Vui lòng lấy access token trước.")
    order_list = []


Lấy đơn hàng từ: 2025-09-27 00:00:00
Đến: 2025-09-27 23:59:59

Đang lấy danh sách đơn hàng...

=== DEBUG: CẤU TRÚC PHẢN HỒI GET_ORDER_LIST ===
Khóa cấp 1: ['error', 'message', 'response', 'request_id']
{
  "error": "",
  "message": "",
  "response": {
    "more": false,
    "next_cursor": "",
    "order_list": [
      {
        "order_sn": "250928M3QYWY3U",
        "booking_sn": ""
      },
      {
        "order_sn": "250928M389BNFJ",
        "booking_sn": ""
      },
      {
        "order_sn": "250928M2QBMP8G",
        "booking_sn": ""
      },
      {
        "order_sn": "250928M2NGJ2NV",
        "booking_sn": ""
      },
      {
        "order_sn": "250928M22KXQBM",
        "booking_sn": ""
      },
      {
        "order_sn": "250928M1QNX38E",
        "booking_sn": ""
      },
      {
        "order_sn": "250928M1H0A5KK",
        "booking_sn": ""
      },
      {
        "order_sn": "250928M0SKN1EF",
        "booking_sn": ""
      },
      {
        "order_sn": "250927M0M70HTW",


## 5.1. Lấy chi tiết đơn hàng (JSON thô) và lưu file

Mục tiêu: sử dụng danh sách `order_sn` từ bước get_order_list để gọi API `get_order_detail`, lưu toàn bộ phản hồi JSON thô ra file để kiểm tra cấu trúc trước khi chuẩn hóa DataFrame.


In [27]:
def fetch_order_detail_batch(partner_id: str, partner_key: str, shop_id: str, access_token: str, order_sn_batch: list) -> dict:
    """
    Gọi API get_order_detail cho tối đa 50 order_sn một lần, trả về JSON thô
    """
    path = '/api/v2/order/get_order_detail'
    timestamp = int(time.time())
    sign = create_signature(str(partner_id), partner_key, path, timestamp, access_token, str(shop_id))

    url = f"https://partner.shopeemobile.com{path}"

    # Query params bắt buộc
    params = {
        'partner_id': int(partner_id),
        'timestamp': timestamp,
        'sign': sign,
        'access_token': access_token,
        'shop_id': int(shop_id),
        'order_sn_list': ','.join(order_sn_batch)
    }

    # Tùy chọn mở rộng thêm field theo tài liệu Shopee
    params['response_optional_fields'] = (
        'buyer_user_id,buyer_username,estimated_shipping_fee,recipient_address,actual_shipping_fee,'
        'goods_to_declare,note,note_update_time,item_list,pay_time,dropshipper,dropshipper_phone,split_up,'
        'buyer_cancel_reason,cancel_by,cancel_reason,actual_shipping_fee_confirmed,buyer_cpf_id,fulfillment_flag,'
        'pickup_done_time,package_list,shipping_carrier,payment_method,total_amount,buyer_username,invoice_data,'
        'order_chargeable_weight_gram,return_request_due_date,edt,payment_info'
    )

    try:
        resp = requests.get(url, params=params)
        data = resp.json()
        if resp.status_code != 200 or data.get('error'):
            print("❌ Lỗi get_order_detail:")
            print(f"  HTTP {resp.status_code}")
            print(f"  error: {data.get('error')}")
            print(f"  message: {data.get('message')}")
            return None
        return data
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi kết nối get_order_detail: {e}")
        return None

# Thu thập order_sn từ order_list
order_sn_list = [o.get('order_sn') for o in order_list] if 'order_list' in locals() else []

# Batch size tối đa 50 theo tài liệu Shopee
batch_size = 50
all_detail_raw = []

if order_sn_list:
    print(f"\nĐang gọi get_order_detail cho {len(order_sn_list)} đơn hàng...")
    for i in range(0, len(order_sn_list), batch_size):
        batch = order_sn_list[i:i+batch_size]
        detail_json = fetch_order_detail_batch(PARTNER_ID, PARTNER_KEY, SHOP_ID, access_token, batch)
        if detail_json:
            all_detail_raw.append(detail_json)
            print(f"  ✓ Batch {i//batch_size + 1}: {len(batch)} order_sn")
        time.sleep(0.5)  # tránh rate limit

    # Lưu JSON thô gộp
    try:
        with open('debug_get_order_detail.json', 'w', encoding='utf-8') as f:
            json.dump(all_detail_raw, f, ensure_ascii=False, indent=2)
        print("\n(Đã lưu JSON thô: debug_get_order_detail.json)")

        # In preview cấu trúc
        preview = json.dumps(all_detail_raw[:1], ensure_ascii=False, indent=2)
        print("\n=== DEBUG: MẪU PHẢN HỒI GET_ORDER_DETAIL (batch đầu) ===")
        print(preview[:8000])
    except Exception as e:
        print(f"❌ Lỗi lưu debug JSON order_detail: {e}")
else:
    print("Không có order_sn để gọi get_order_detail")



Đang gọi get_order_detail cho 98 đơn hàng...
  ✓ Batch 1: 50 order_sn
  ✓ Batch 2: 48 order_sn

(Đã lưu JSON thô: debug_get_order_detail.json)

=== DEBUG: MẪU PHẢN HỒI GET_ORDER_DETAIL (batch đầu) ===
[
  {
    "error": "",
    "message": "",
    "response": {
      "order_list": [
        {
          "actual_shipping_fee": 0,
          "actual_shipping_fee_confirmed": true,
          "advance_package": false,
          "booking_sn": "",
          "buyer_cancel_reason": "Need to input / Change Voucher Code",
          "buyer_cpf_id": null,
          "buyer_user_id": 951993682,
          "buyer_username": "dien_282",
          "cancel_by": "buyer",
          "cancel_reason": "Need to input / Change Voucher Code",
          "cod": true,
          "create_time": 1758992022,
          "currency": "VND",
          "days_to_ship": 2,
          "dropshipper": null,
          "dropshipper_phone": null,
          "edt_from": 0,
          "edt_to": 0,
          "estimated_shipping_fee": 43900,


## 6. Xử lý dữ liệu JSON thành các DataFrame theo thiết kế ERD

Mục tiêu: Chuyển đổi dữ liệu JSON từ `debug_get_order_detail.json` thành các bảng DataFrame theo sơ đồ quan hệ đã thiết kế và xuất ra file Excel với nhiều sheet.


In [28]:
# Đọc dữ liệu JSON từ file debug
try:
    with open('debug_get_order_detail.json', 'r', encoding='utf-8') as f:
        detail_batches = json.load(f)
    
    # Gộp tất cả order_list từ các batch thành một list duy nhất
    orders_detail_all = []
    for batch in detail_batches:
        orders_detail_all.extend(batch.get('response', {}).get('order_list', []))
    
    print(f"✅ Đã đọc {len(orders_detail_all)} đơn hàng từ JSON detail")
    
    # Lấy request_id từ batch đầu tiên để làm source_request_id
    source_request_id = detail_batches[0].get('request_id', '') if detail_batches else ''
    print(f"Source Request ID: {source_request_id}")
    
except Exception as e:
    print(f"❌ Lỗi đọc file JSON: {e}")
    orders_detail_all = []
    source_request_id = ''


✅ Đã đọc 98 đơn hàng từ JSON detail
Source Request ID: e3e3e7f33fd9a3d07aab659083234f00:010002960565c8b1:000000c3cc198fc4


In [29]:
# Hàm chuyển đổi timestamp Unix thành datetime
def unix_to_datetime(unix_timestamp):
    """Chuyển đổi Unix timestamp thành datetime, trả về None nếu timestamp = 0 hoặc None"""
    if not unix_timestamp or unix_timestamp == 0:
        return None
    try:
        return pd.to_datetime(unix_timestamp, unit='s', utc=True)
    except:
        return None

# Hàm xử lý dữ liệu orders
def process_orders_data(orders_list, source_request_id):
    """Xử lý dữ liệu orders từ JSON thành DataFrame"""
    orders_data = []
    
    for order in orders_list:
        order_data = {
            'order_sn': order.get('order_sn'),
            'region': order.get('region'),
            'currency': order.get('currency'),
            'cod': order.get('cod'),
            'total_amount': order.get('total_amount'),
            'order_status': order.get('order_status'),
            'shipping_carrier': order.get('shipping_carrier'),
            'payment_method': order.get('payment_method'),
            'estimated_shipping_fee': order.get('estimated_shipping_fee'),
            'message_to_seller': order.get('message_to_seller'),
            'create_time': unix_to_datetime(order.get('create_time')),
            'update_time': unix_to_datetime(order.get('update_time')),
            'days_to_ship': order.get('days_to_ship'),
            'ship_by_date': unix_to_datetime(order.get('ship_by_date')),
            'buyer_user_id': order.get('buyer_user_id'),
            'buyer_username': order.get('buyer_username'),
            
            # Optional fields
            'actual_shipping_fee': order.get('actual_shipping_fee'),
            'actual_shipping_fee_confirmed': order.get('actual_shipping_fee_confirmed'),
            'goods_to_declare': order.get('goods_to_declare'),
            'note': order.get('note'),
            'note_update_time': unix_to_datetime(order.get('note_update_time')),
            'pay_time': unix_to_datetime(order.get('pay_time')),
            'dropshipper': order.get('dropshipper'),
            'dropshipper_phone': order.get('dropshipper_phone'),
            'split_up': order.get('split_up'),
            'buyer_cancel_reason': order.get('buyer_cancel_reason'),
            'cancel_by': order.get('cancel_by'),
            'cancel_reason': order.get('cancel_reason'),
            'buyer_cpf_id': order.get('buyer_cpf_id'),
            'fulfillment_flag': order.get('fulfillment_flag'),
            'pickup_done_time': unix_to_datetime(order.get('pickup_done_time')),
            'reverse_shipping_fee': order.get('reverse_shipping_fee'),
            'order_chargeable_weight_gram': order.get('order_chargeable_weight_gram'),
            'prescription_check_status': order.get('prescription_check_status'),
            'pharmacist_name': order.get('pharmacist_name'),
            'prescription_approval_time': unix_to_datetime(order.get('prescription_approval_time')),
            'prescription_rejection_time': unix_to_datetime(order.get('prescription_rejection_time')),
            'edt_from': unix_to_datetime(order.get('edt_from')),
            'edt_to': unix_to_datetime(order.get('edt_to')),
            'booking_sn': order.get('booking_sn'),
            'advance_package': order.get('advance_package'),
            'return_request_due_date': unix_to_datetime(order.get('return_request_due_date')),
            'is_buyer_shop_collection': order.get('is_buyer_shop_collection'),
            
            # Meta fields
            'source_request_id': source_request_id,
            'ingested_at': pd.Timestamp.now(tz='UTC')
        }
        orders_data.append(order_data)
    
    return pd.DataFrame(orders_data)

# Xử lý dữ liệu orders
if orders_detail_all:
    print("Đang xử lý dữ liệu orders...")
    orders_df = process_orders_data(orders_detail_all, source_request_id)
    print(f"✅ Đã tạo orders_df với {len(orders_df)} dòng")
    print(f"Các cột: {list(orders_df.columns)}")
else:
    print("❌ Không có dữ liệu để xử lý")
    orders_df = pd.DataFrame()


Đang xử lý dữ liệu orders...
✅ Đã tạo orders_df với 98 dòng
Các cột: ['order_sn', 'region', 'currency', 'cod', 'total_amount', 'order_status', 'shipping_carrier', 'payment_method', 'estimated_shipping_fee', 'message_to_seller', 'create_time', 'update_time', 'days_to_ship', 'ship_by_date', 'buyer_user_id', 'buyer_username', 'actual_shipping_fee', 'actual_shipping_fee_confirmed', 'goods_to_declare', 'note', 'note_update_time', 'pay_time', 'dropshipper', 'dropshipper_phone', 'split_up', 'buyer_cancel_reason', 'cancel_by', 'cancel_reason', 'buyer_cpf_id', 'fulfillment_flag', 'pickup_done_time', 'reverse_shipping_fee', 'order_chargeable_weight_gram', 'prescription_check_status', 'pharmacist_name', 'prescription_approval_time', 'prescription_rejection_time', 'edt_from', 'edt_to', 'booking_sn', 'advance_package', 'return_request_due_date', 'is_buyer_shop_collection', 'source_request_id', 'ingested_at']


In [30]:
# Hàm xử lý dữ liệu recipient_address
def process_recipient_address_data(orders_list, source_request_id):
    """Xử lý dữ liệu recipient_address từ JSON thành DataFrame"""
    address_data = []
    
    for order in orders_list:
        order_sn = order.get('order_sn')
        recipient_address = order.get('recipient_address')
        
        if recipient_address:
            address_data.append({
                'order_sn': order_sn,
                'name': recipient_address.get('name'),
                'phone': recipient_address.get('phone'),
                'town': recipient_address.get('town'),
                'district': recipient_address.get('district'),
                'city': recipient_address.get('city'),
                'state': recipient_address.get('state'),
                'region': recipient_address.get('region'),
                'zipcode': recipient_address.get('zipcode'),
                'full_address': recipient_address.get('full_address'),
                'latitude': recipient_address.get('geolocation', {}).get('latitude') if recipient_address.get('geolocation') else None,
                'longitude': recipient_address.get('geolocation', {}).get('longitude') if recipient_address.get('geolocation') else None,
                'source_request_id': source_request_id,
                'ingested_at': pd.Timestamp.now(tz='UTC')
            })
    
    return pd.DataFrame(address_data)

# Xử lý dữ liệu recipient_address
if orders_detail_all:
    print("Đang xử lý dữ liệu recipient_address...")
    recipient_address_df = process_recipient_address_data(orders_detail_all, source_request_id)
    print(f"✅ Đã tạo recipient_address_df với {len(recipient_address_df)} dòng")
else:
    recipient_address_df = pd.DataFrame()


Đang xử lý dữ liệu recipient_address...
✅ Đã tạo recipient_address_df với 98 dòng


In [31]:
# Hàm xử lý dữ liệu order_items và order_item_locations
def process_order_items_data(orders_list, source_request_id):
    """Xử lý dữ liệu order_items và order_item_locations từ JSON thành DataFrame"""
    items_data = []
    item_locations_data = []
    
    for order in orders_list:
        order_sn = order.get('order_sn')
        item_list = order.get('item_list', [])
        
        for item in item_list:
            # Xử lý order_items
            item_data = {
                'order_sn': order_sn,
                'order_item_id': item.get('order_item_id'),
                'item_id': item.get('item_id'),
                'item_name': item.get('item_name'),
                'item_sku': item.get('item_sku'),
                'model_id': item.get('model_id'),
                'model_name': item.get('model_name'),
                'model_sku': item.get('model_sku'),
                'model_quantity_purchased': item.get('model_quantity_purchased'),
                'model_original_price': item.get('model_original_price'),
                'model_discounted_price': item.get('model_discounted_price'),
                'wholesale': item.get('wholesale'),
                'weight': item.get('weight'),
                'add_on_deal': item.get('add_on_deal'),
                'main_item': item.get('main_item'),
                'add_on_deal_id': item.get('add_on_deal_id'),
                'promotion_type': item.get('promotion_type'),
                'promotion_id': item.get('promotion_id'),
                'promotion_group_id': item.get('promotion_group_id'),
                'is_prescription_item': item.get('is_prescription_item'),
                'is_b2c_owned_item': item.get('is_b2c_owned_item'),
                'consultation_id': item.get('consultation_id'),
                'image_url': item.get('image_info', {}).get('image_url') if item.get('image_info') else None,
                'source_request_id': source_request_id,
                'ingested_at': pd.Timestamp.now(tz='UTC')
            }
            items_data.append(item_data)
            
            # Xử lý order_item_locations
            product_location_id = item.get('product_location_id', [])
            if isinstance(product_location_id, list):
                for location_id in product_location_id:
                    item_locations_data.append({
                        'order_sn': order_sn,
                        'order_item_id': item.get('order_item_id'),
                        'location_id': location_id,
                        'source_request_id': source_request_id,
                        'ingested_at': pd.Timestamp.now(tz='UTC')
                    })
            elif isinstance(product_location_id, str):
                item_locations_data.append({
                    'order_sn': order_sn,
                    'order_item_id': item.get('order_item_id'),
                    'location_id': product_location_id,
                    'source_request_id': source_request_id,
                    'ingested_at': pd.Timestamp.now(tz='UTC')
                })
    
    return pd.DataFrame(items_data), pd.DataFrame(item_locations_data)

# Xử lý dữ liệu order_items và order_item_locations
if orders_detail_all:
    print("Đang xử lý dữ liệu order_items và order_item_locations...")
    order_items_df, order_item_locations_df = process_order_items_data(orders_detail_all, source_request_id)
    print(f"✅ Đã tạo order_items_df với {len(order_items_df)} dòng")
    print(f"✅ Đã tạo order_item_locations_df với {len(order_item_locations_df)} dòng")
else:
    order_items_df = pd.DataFrame()
    order_item_locations_df = pd.DataFrame()


Đang xử lý dữ liệu order_items và order_item_locations...
✅ Đã tạo order_items_df với 128 dòng
✅ Đã tạo order_item_locations_df với 128 dòng


In [32]:
# Hàm xử lý dữ liệu packages và package_items
def process_packages_data(orders_list, source_request_id):
    """Xử lý dữ liệu packages và package_items từ JSON thành DataFrame"""
    packages_data = []
    package_items_data = []
    
    for order in orders_list:
        order_sn = order.get('order_sn')
        package_list = order.get('package_list', [])
        
        for package in package_list:
            # Xử lý packages
            package_data = {
                'order_sn': order_sn,
                'package_number': package.get('package_number'),
                'logistics_status': package.get('logistics_status'),
                'logistics_channel_id': package.get('logistics_channel_id'),
                'shipping_carrier': package.get('shipping_carrier'),
                'allow_self_design_awb': package.get('allow_self_design_awb'),
                'parcel_chargeable_weight_gram': package.get('parcel_chargeable_weight_gram'),
                'group_shipment_id': package.get('group_shipment_id'),
                'virtual_contact_number': package.get('virtual_contact_number'),
                'package_query_number': package.get('package_query_number'),
                'sorting_group': package.get('sorting_group'),
                'source_request_id': source_request_id,
                'ingested_at': pd.Timestamp.now(tz='UTC')
            }
            packages_data.append(package_data)
            
            # Xử lý package_items
            package_item_list = package.get('item_list', [])
            for pkg_item in package_item_list:
                package_items_data.append({
                    'order_sn': order_sn,
                    'package_number': package.get('package_number'),
                    'order_item_id': pkg_item.get('order_item_id'),
                    'item_id': pkg_item.get('item_id'),
                    'model_id': pkg_item.get('model_id'),
                    'model_quantity': pkg_item.get('model_quantity'),
                    'promotion_group_id': pkg_item.get('promotion_group_id'),
                    'product_location_id': pkg_item.get('product_location_id'),
                    'parcel_chargeable_weight': pkg_item.get('parcel_chargeable_weight'),
                    'source_request_id': source_request_id,
                    'ingested_at': pd.Timestamp.now(tz='UTC')
                })
    
    return pd.DataFrame(packages_data), pd.DataFrame(package_items_data)

# Xử lý dữ liệu packages và package_items
if orders_detail_all:
    print("Đang xử lý dữ liệu packages và package_items...")
    packages_df, package_items_df = process_packages_data(orders_detail_all, source_request_id)
    print(f"✅ Đã tạo packages_df với {len(packages_df)} dòng")
    print(f"✅ Đã tạo package_items_df với {len(package_items_df)} dòng")
else:
    packages_df = pd.DataFrame()
    package_items_df = pd.DataFrame()


Đang xử lý dữ liệu packages và package_items...
✅ Đã tạo packages_df với 98 dòng
✅ Đã tạo package_items_df với 128 dòng


In [36]:
# Hàm xử lý dữ liệu invoice
def process_invoice_data(orders_list, source_request_id):
    """Xử lý dữ liệu invoice từ JSON thành DataFrame"""
    invoice_data = []
    
    for order in orders_list:
        order_sn = order.get('order_sn')
        invoice_data_obj = order.get('invoice_data')
        
        if invoice_data_obj:
            invoice_data.append({
                'order_sn': order_sn,
                'number': invoice_data_obj.get('number'),
                'series_number': invoice_data_obj.get('series_number'),
                'access_key': invoice_data_obj.get('access_key'),
                'issue_date': unix_to_datetime(invoice_data_obj.get('issue_date')),
                'total_value': invoice_data_obj.get('total_value'),
                'products_total_value': invoice_data_obj.get('products_total_value'),
                'tax_code': invoice_data_obj.get('tax_code'),
                'source_request_id': source_request_id,
                'ingested_at': pd.Timestamp.now(tz='UTC')
            })
    
    return pd.DataFrame(invoice_data)

# Hàm xử lý dữ liệu payment_info
def process_payment_info_data(orders_list, source_request_id):
    """Xử lý dữ liệu payment_info từ JSON thành DataFrame"""
    payment_info_data = []
    
    for order in orders_list:
        order_sn = order.get('order_sn')
        payment_info_list = order.get('payment_info', [])
        
        # Kiểm tra nếu payment_info_list không phải None và là danh sách
        if payment_info_list and isinstance(payment_info_list, list):
            for payment in payment_info_list:
                payment_info_data.append({
                    'order_sn': order_sn,
                    'transaction_id': payment.get('transaction_id'),
                    'payment_method': payment.get('payment_method'),
                    'payment_processor_register': payment.get('payment_processor_register'),
                    'card_brand': payment.get('card_brand'),
                    'source_request_id': source_request_id,
                    'ingested_at': pd.Timestamp.now(tz='UTC')
                })
    
    return pd.DataFrame(payment_info_data)

# Xử lý dữ liệu invoice và payment_info
if orders_detail_all:
    print("Đang xử lý dữ liệu invoice và payment_info...")
    invoice_df = process_invoice_data(orders_detail_all, source_request_id)
    payment_info_df = process_payment_info_data(orders_detail_all, source_request_id)
    print(f"✅ Đã tạo invoice_df với {len(invoice_df)} dòng")
    print(f"✅ Đã tạo payment_info_df với {len(payment_info_df)} dòng")
else:
    invoice_df = pd.DataFrame()
    payment_info_df = pd.DataFrame()


Đang xử lý dữ liệu invoice và payment_info...
✅ Đã tạo invoice_df với 0 dòng
✅ Đã tạo payment_info_df với 0 dòng


In [38]:
# Hàm xử lý dữ liệu các mảng đơn giản
def process_array_data(orders_list, source_request_id):
    """Xử lý dữ liệu các mảng đơn giản từ JSON thành DataFrame"""
    pending_terms_data = []
    warnings_data = []
    prescription_images_data = []
    buyer_proof_data = []
    
    for order in orders_list:
        order_sn = order.get('order_sn')
        
        # pending_terms
        pending_terms = order.get('pending_terms', [])
        if pending_terms and isinstance(pending_terms, list):
            for term in pending_terms:
                pending_terms_data.append({
                    'order_sn': order_sn,
                    'term': term,
                    'source_request_id': source_request_id,
                    'ingested_at': pd.Timestamp.now(tz='UTC')
                })
        
        # warnings
        warnings = order.get('warnings', [])
        if warnings and isinstance(warnings, list):
            for warning in warnings:
                warnings_data.append({
                    'order_sn': order_sn,
                    'warning': warning,
                    'source_request_id': source_request_id,
                    'ingested_at': pd.Timestamp.now(tz='UTC')
                })
        
        # prescription_images
        prescription_images = order.get('prescription_images', [])
        if prescription_images and isinstance(prescription_images, list):
            for image_url in prescription_images:
                prescription_images_data.append({
                    'order_sn': order_sn,
                    'image_url': image_url,
                    'source_request_id': source_request_id,
                    'ingested_at': pd.Timestamp.now(tz='UTC')
                })
        
        # buyer_proof_of_collection (nếu có)
        buyer_proof = order.get('buyer_proof_of_collection', [])
        if buyer_proof and isinstance(buyer_proof, list):
            for image_url in buyer_proof:
                buyer_proof_data.append({
                    'order_sn': order_sn,
                    'image_url': image_url,
                    'source_request_id': source_request_id,
                    'ingested_at': pd.Timestamp.now(tz='UTC')
                })
    
    return (pd.DataFrame(pending_terms_data), 
            pd.DataFrame(warnings_data), 
            pd.DataFrame(prescription_images_data),
            pd.DataFrame(buyer_proof_data))

# Xử lý dữ liệu các mảng đơn giản
if orders_detail_all:
    print("Đang xử lý dữ liệu các mảng đơn giản...")
    pending_terms_df, warnings_df, prescription_images_df, buyer_proof_df = process_array_data(orders_detail_all, source_request_id)
    print(f"✅ Đã tạo pending_terms_df với {len(pending_terms_df)} dòng")
    print(f"✅ Đã tạo warnings_df với {len(warnings_df)} dòng")
    print(f"✅ Đã tạo prescription_images_df với {len(prescription_images_df)} dòng")
    print(f"✅ Đã tạo buyer_proof_df với {len(buyer_proof_df)} dòng")
else:
    pending_terms_df = pd.DataFrame()
    warnings_df = pd.DataFrame()
    prescription_images_df = pd.DataFrame()
    buyer_proof_df = pd.DataFrame()


Đang xử lý dữ liệu các mảng đơn giản...
✅ Đã tạo pending_terms_df với 0 dòng
✅ Đã tạo warnings_df với 0 dòng
✅ Đã tạo prescription_images_df với 0 dòng
✅ Đã tạo buyer_proof_df với 0 dòng


In [41]:
# Hàm chuyển đổi datetime có timezone thành timezone-naive
def convert_datetime_to_naive(df):
    """Chuyển đổi tất cả datetime columns từ timezone-aware sang timezone-naive"""
    df_copy = df.copy()
    
    for col in df_copy.columns:
        if df_copy[col].dtype == 'datetime64[ns, UTC]':
            # Chuyển từ UTC về local time và loại bỏ timezone info
            df_copy[col] = df_copy[col].dt.tz_localize(None)
        elif 'datetime' in str(df_copy[col].dtype):
            # Xử lý các datetime columns khác
            df_copy[col] = pd.to_datetime(df_copy[col], utc=True).dt.tz_localize(None)
    
    return df_copy

# Xuất dữ liệu ra file Excel với nhiều sheet
def export_to_excel_with_sheets(dataframes_dict, filename):
    """Xuất các DataFrame ra file Excel với mỗi sheet là một bảng"""
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            for sheet_name, df in dataframes_dict.items():
                if not df.empty:
                    # Chuyển đổi datetime columns trước khi xuất
                    df_export = convert_datetime_to_naive(df)
                    df_export.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"✅ Đã xuất {sheet_name}: {len(df)} dòng")
                else:
                    # Tạo sheet trống với header
                    empty_df = pd.DataFrame(columns=df.columns if hasattr(df, 'columns') else [])
                    empty_df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"⚠️ Sheet {sheet_name} trống")
        
        print(f"\\n🎉 Đã xuất thành công file Excel: {filename}")
        return True
    except Exception as e:
        print(f"❌ Lỗi xuất file Excel: {e}")
        return False

# Tạo dictionary chứa tất cả các DataFrame
all_dataframes = {
    'orders': orders_df,
    'recipient_address': recipient_address_df,
    'order_items': order_items_df,
    'order_item_locations': order_item_locations_df,
    'packages': packages_df,
    'package_items': package_items_df,
    'invoice': invoice_df,
    'payment_info': payment_info_df,
    'order_pending_terms': pending_terms_df,
    'order_warnings': warnings_df,
    'prescription_images': prescription_images_df,
    'buyer_proof_of_collection': buyer_proof_df
}

# Tạo tên file với timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
excel_filename = f"shopee_orders_data_{timestamp}.xlsx"

# Xuất ra file Excel
print("\\n" + "="*50)
print("XUẤT DỮ LIỆU RA FILE EXCEL")
print("="*50)

export_success = export_to_excel_with_sheets(all_dataframes, excel_filename)

if export_success:
    print(f"\\n📊 Tóm tắt dữ liệu đã xuất:")
    for name, df in all_dataframes.items():
        print(f"  {name}: {len(df)} dòng")
else:
    print("\\n❌ Xuất file Excel thất bại")


\n==================================================
XUẤT DỮ LIỆU RA FILE EXCEL
✅ Đã xuất orders: 98 dòng
✅ Đã xuất recipient_address: 98 dòng
✅ Đã xuất order_items: 128 dòng
✅ Đã xuất order_item_locations: 128 dòng
✅ Đã xuất packages: 98 dòng
✅ Đã xuất package_items: 128 dòng
⚠️ Sheet invoice trống
⚠️ Sheet payment_info trống
⚠️ Sheet order_pending_terms trống
⚠️ Sheet order_warnings trống
⚠️ Sheet prescription_images trống
⚠️ Sheet buyer_proof_of_collection trống
\n🎉 Đã xuất thành công file Excel: shopee_orders_data_20250928_173039.xlsx
\n📊 Tóm tắt dữ liệu đã xuất:
  orders: 98 dòng
  recipient_address: 98 dòng
  order_items: 128 dòng
  order_item_locations: 128 dòng
  packages: 98 dòng
  package_items: 128 dòng
  invoice: 0 dòng
  payment_info: 0 dòng
  order_pending_terms: 0 dòng
  order_warnings: 0 dòng
  prescription_images: 0 dòng
  buyer_proof_of_collection: 0 dòng
